<a href="https://colab.research.google.com/github/Shoumik29/Parallel-Processing-and-Distributed-System-Lab/blob/main/CUDA_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Jan 19 18:49:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-krg0y5j7
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-krg0y5j7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define K 2
#define M 4
#define N 4
#define P 4

void init_mat(int *mat, int a, int b){
    for(int k=0;k<K;k++){
        for(int i=0;i<a*b;i++) mat[k*M*N+i] = rand()%10;
    }
}

void print_mat(int *mat, int a, int b, int indx){
    for(int i=0;i<a;i++){
        for(int j=0;j<b;j++) printf("%d ", mat[(indx*K)+(i*b)+j]);
        printf("\n");
    }
}

//kernel
__global__ void matmul(int *a, int *b, int *c){

    int z = blockIdx.x;
    int x = threadIdx.x;
    int y = threadIdx.y;

    int c1 = M;
    int c2 = P;
    int c3 = K;

    c[(z*c3)+(x*c2)+y] = 0;
    for(int q=0;q<c1;q++){
      c[(z*c3)+(x*c2)+y] = c[(z*c3)+(x*c2)+y] + (a[(z*c3)+(x*c2)+q] * b[(z*c3)+(q*c2)+y]);
    }
}


int main(){

    int mat_a[K*M*N];
    int mat_b[K*N*P];
    int mat_c[K*M*P];

    //Initializing matrices
    init_mat(mat_a, M, N);
    init_mat(mat_b, N, P);

    print_mat(mat_a, M, N, 1);
    printf("\n");
    print_mat(mat_b, N, P, 1);
    printf("\n");

    //device memory
    int* d_a;
    int* d_b;
    int* d_c;

    cudaMalloc((void**)&d_a, K*M*N*sizeof(int));
    cudaMalloc((void**)&d_b, K*N*P*sizeof(int));
    cudaMalloc((void**)&d_c, K*M*P*sizeof(int));

    cudaMemcpy(d_a, mat_a, K*M*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, mat_b, K*N*P*sizeof(int), cudaMemcpyHostToDevice);

    dim3 grid_size(K);
    dim3 block_size(M,P);

    //Launch kernel
    matmul<<<grid_size, block_size>>>(d_a, d_b, d_c);

    cudaMemcpy(mat_c, d_c, K*M*P*sizeof(int), cudaMemcpyDeviceToHost);

    print_mat(mat_c, M, P, 1);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;

}

7 5 3 5 
6 2 9 1 
2 7 0 9 
3 6 0 6 

9 7 3 6 
1 2 9 3 
1 9 4 7 
8 4 5 0 

111 106 103 78 
73 131 77 105 
97 64 114 33 
81 57 93 36 



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <cuda_runtime.h>
#define T 1000
#define S 50


typedef struct {
    char name[S];
    char phone[S];
} Contact;


//kernel
__global__ void search_contact(Contact* contacts, const char* searchName, int* indx){

    int i = threadIdx.x;
    int st_len, searchName_len=0, contact_len=0;
    for(int j=0;searchName[j]!='\0';j++) searchName_len++;
    for(int j=0;contacts[i].name[j]!='\0';j++) contact_len++;
    if(searchName_len <= contact_len) st_len = searchName_len;
    else st_len = contact_len;
    int flag = 1;
    for(int j=0;j<st_len;j++){
        if(searchName[j] != contacts[i].name[j]){
            flag = 0;
            break;
        }
    }
    if(searchName_len != contact_len) flag = 0;
    if(flag==1){
        indx[i] = i;
    }
    else{
        indx[i] = -1;
    }
}


int main(){

    Contact contacts[T];
    char searchName[S] = "Grace Taylor";

    const char* phonebookFile_1 = "/content/sample_data/input.txt";
    const char* phonebookFile_2 = "/content/sample_data/input_1.txt";

    int total_contacts = 0;

    FILE* file_1 = fopen(phonebookFile_1, "r");
    FILE* file_2 = fopen(phonebookFile_2, "r");

    while(fscanf(file_1, "%49[^,],%111[^\n]\n", contacts[total_contacts].name, contacts[total_contacts].phone)!= EOF){
        //printf("%s    %s\n", contacts[total_contacts].name, contacts[total_contacts].phone);
        total_contacts += 1;
    }
    while(fscanf(file_2, "%49[^,],%111[^\n]\n", contacts[total_contacts].name, contacts[total_contacts].phone)!= EOF){
        //printf("%s    %s\n", contacts[total_contacts].name, contacts[total_contacts].phone);
        total_contacts += 1;
    }

    fclose(file_1);
    fclose(file_2);

    //device memory
    Contact* d_contacts;
    char* d_searchName;
    int* d_indx;

    int t = total_contacts;

    cudaMalloc((void**)&d_contacts, sizeof(Contact)*t);
    cudaMalloc((void**)&d_searchName, S*sizeof(char));
    cudaMalloc((void**)&d_indx, sizeof(int)*t);

    cudaMemcpy(d_contacts, contacts, sizeof(Contact)*t, cudaMemcpyHostToDevice);
    cudaMemcpy(d_searchName, searchName, sizeof(char)*S, cudaMemcpyHostToDevice);

    dim3 grid_size(1);
    dim3 block_size(t);

    search_contact<<<grid_size, block_size>>>(d_contacts, d_searchName, d_indx);

    int indx[t]={0};
    cudaMemcpy(indx, d_indx, sizeof(int)*t, cudaMemcpyDeviceToHost);

    int nf = 0;
    for(int y=0;y<t;y++){
      if(indx[y]>=0){
        nf = 0;
        printf("%s  %s\n", contacts[indx[y]].name, contacts[indx[y]].phone);
        break;
      }
      else nf = 1;
    }
    if(nf) printf("Not Found\n");

    cudaFree(d_contacts);
    cudaFree(d_indx);

    return 0;
}

Grace Taylor  7890123456

